In [12]:
import re
import subprocess
from pathlib import Path
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import theme
from collections import OrderedDict  

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
p = Path('logs_thresh')
directories = [x.name for x in p.iterdir() if x.is_dir()]
lines_dict = {}

In [6]:
def tail(f, n, offset=0):
    proc = subprocess.Popen(['tail', '-n', str(n + offset), f], stdout=subprocess.PIPE)
    lines = proc.stdout.readlines()
    return lines[:offset]

def return_result(f):
    result = [x.decode("utf-8") for x in tail(f, 7, 2)]
    result = [re.findall('\d+', result[0])[0]] + re.findall('\d+', result[1])
    result = [float(x) for x in result] # length of 7, [deg, wt pz/1-pz, 0.1, 0.3, 0.5, 0.7, 1.0]
    return result

In [7]:
for dir in directories:
    [N,K,l,s,n,exact] = [''.join(x for x in r if x.isdigit()) for r in dir.split('_')[:-1]]
    con = dir.split('_')[-1]
    exp_name = 'N='+N+',K='+K+',l='+l+',n='+n+','+con
    print(exp_name)
    q = p / dir
    files = list(q.glob('*.log'))
    pzs = [''.join(x for x in f.name if x.isdigit()) for f in files]
    pzs = np.array([float(pz) for pz in pzs]) / 1000.0
    results = [return_result(f) for f in files]
    deg_err = np.array([x[0] for x in results]) / float(n)
    true_weight_err = np.array([x[1] for x in results]) / float(n)
    # sort so that the line will be smooth
    order = np.argsort(pzs)
    pzs = pzs[order]
    deg_err = deg_err[order]
    true_weight_err = true_weight_err[order]
    exp_name_deg = exp_name + ',deg'
    exp_name_wt = exp_name + ',wt'
    if exp_name_deg not in lines_dict:
        lines_dict[exp_name_deg]= (pzs, [deg_err])
        lines_dict[exp_name_wt] = (pzs, [true_weight_err])
    else:
        lines_dict[exp_name_deg][1].append(deg_err)
        lines_dict[exp_name_wt][1].append(true_weight_err)

N=1024,K=513,l=128,n=10000,PW
N=128,K=65,l=128,n=10000,PW
N=2048,K=1025,l=128,n=10000,PW
N=512,K=257,l=128,n=10000,PW
N=512,K=257,l=128,n=10000,PW
N=2048,K=1025,l=128,n=10000,PW
N=64,K=33,l=128,n=10000,PW
N=128,K=65,l=128,n=10000,PW
N=256,K=129,l=128,n=10000,PW
N=256,K=129,l=128,n=10000,PW
N=64,K=33,l=128,n=10000,PW
N=128,K=65,l=128,n=10000,PW
N=1024,K=513,l=128,n=10000,PW
N=64,K=33,l=128,n=10000,PW
N=256,K=129,l=128,n=10000,PW
N=64,K=33,l=128,n=10000,PW
N=512,K=257,l=128,n=10000,PW
N=256,K=129,l=128,n=10000,PW
N=128,K=65,l=128,n=10000,PW
N=2048,K=1025,l=128,n=10000,PW
N=1024,K=513,l=128,n=10000,PW
N=128,K=65,l=128,n=10000,PW
N=64,K=33,l=128,n=10000,PW
N=2048,K=1025,l=128,n=10000,PW
N=1024,K=513,l=128,n=10000,PW
N=1024,K=513,l=128,n=10000,PW
N=256,K=129,l=128,n=10000,PW
N=256,K=129,l=128,n=10000,PW
N=512,K=257,l=128,n=10000,PW
N=128,K=65,l=128,n=10000,PW
N=1024,K=513,l=128,n=10000,PW
N=1024,K=513,l=128,n=10000,PW
N=2048,K=1025,l=128,n=10000,PW
N=256,K=129,l=128,n=10000,PW
N=64,K=33,l=1

In [9]:
data_dict = {}
for key, value in lines_dict.items():
    mean = np.mean(value[1], axis=0)
    var = np.std(value[1], axis=0)
    data_dict[key] = (value[0], mean, var)
    

In [7]:
fig = go.Figure()

for x,y in data_dict.items():
    if ('PW' in x):
        fig.add_scatter(x=y[0], y=y[1], error_y=dict(type='data', array=y[2], visible=True), name=x)

fig.show()
fig.write_html("PW_thresh.html")

In [10]:
color_dict = { '64': 'gold', '128': 'limegreen', '256': 'teal', '512': 'dodgerblue', '1024': 'blue', '2048': 'blueviolet', '4096': 'orchid'}
data_dict = OrderedDict(sorted(data_dict.items(), key=lambda item: int(item[0].split(',')[0].split('=')[-1])))

In [13]:
fig = go.Figure()
fig.add_scatter(x=[0.1], y=[0], mode='lines', name='N', line_color='rgba(255, 255, 255, 0)', showlegend=True)
for x,y in data_dict.items():
    if ('RM' in x) or ('HPW' in x) or ('l=1,' in x):
        continue
    if ('deg' in x):
        print(x)
        l = x.split(',')[0]
        fig.add_scatter(x=y[0], y=y[1], error_y=dict(type='data', array=y[2], visible=True), name=l[2:], line_color=color_dict[l.split('=')[1]], line_dash='solid',marker_size=1)
    else:
        l = x.split(',')[0]
        fig.add_scatter(x=y[0], y=y[1], error_y=dict(type='data', array=y[2], visible=True), name=l[2:], line_color=color_dict[l.split('=')[1]], line_dash='dot', showlegend=False, marker_size=1)

# fig.add_scatter(x=[0.1], y=[0], mode='lines', name='Method', line_color='rgba(255, 255, 255, 0)', showlegend=True)
# fig.add_scatter(x=[0.1], y=[0], mode='lines', name='Bad', line_color='black', showlegend=True)
# fig.add_scatter(x=[0.1], y=[0], mode='lines', name='Good', line_color='black', line_dash='dash', showlegend=True)

fig.update_layout(legend=dict(xanchor='right', yanchor='bottom', x=0.99, y=0.01, orientation='v'), height=800, width=1200)
# fig.update_xaxes(title_text=r'$\sqrt{(n_\text{c}(t|{T_\text{early}}))}$')
# fig.update_xaxes(title_text=r"$\Huge{\text{Physical error rate  } p}$", title_font_size=100)
# fig.update_yaxes(title_text=r'$\Huge{\text{Logical error rate  } p_L}$')
fig.update_xaxes(title_text='Physical error rate')
fig.update_yaxes(title_text='Logical error rate')
fig.show()
fig.write_image("PW_thresh.pdf")

N=64,K=33,l=128,n=10000,PW,deg
N=128,K=65,l=128,n=10000,PW,deg
N=256,K=129,l=128,n=10000,PW,deg
N=512,K=257,l=128,n=10000,PW,deg
N=1024,K=513,l=128,n=10000,PW,deg
N=2048,K=1025,l=128,n=10000,PW,deg
